In [1]:
import geemap
import ee
from datetime import datetime

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize()

In [4]:
aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
aoi = aoi.filter(ee.Filter.eq('country_na', 'Belize'))
aoi = geemap.shp_to_ee('VCS2250/Calyx_VCS_2250_project.shp')

No module named 'pycrs'


In [5]:
aoi = geemap.shp_to_ee('VCS2250/Calyx_VCS_2250_project.shp')
year = 2015

year_string = str(year)
extent_raster = ee.ImageCollection("projects/earthengine-legacy/assets/projects/sat-io/open-datasets/GMW/extent/GMW_V3");
extent_year = extent_raster.filterDate(year_string + '-01-01', year_string + '-12-31').first().clip(aoi)
mangrove_year = extent_year.eq(1)
m2 = geemap.Map()
m2.centerObject(aoi)
m2.addLayer(ee.Image().paint(aoi, 0, 2),{}, 'AOI')
m2.addLayer(mangrove_year.updateMask(mangrove_year), {'palette': ['white', 'green']}, 'Mangrove 1996')
m2

No module named 'pycrs'


Exception: ee_object must be one of ee.Geometry, ee.FeatureCollection, ee.Image, or ee.ImageCollection.

## GMW section

In [17]:
#This includes work from Tao's pre-existing Blue_Carbon_Analysis.ipync code!
def GMWHectares(aoi, year):
    '''
    Inputs: aoi (image, imageCollection, featureCollection). Most importantly, NOT a shpfile. That translation must be done outside of this function.
            year - the year you are looking for, as an integer.
    Output: The area of the aoi covered by mangroves in the given year.
    
    '''
    year_string = str(year)
    extent_raster = ee.ImageCollection("projects/earthengine-legacy/assets/projects/sat-io/open-datasets/GMW/extent/GMW_V3")
    
    #Filter by year, clip to AOI:
    extent_year_clipped = extent_raster.filterDate(year_string + '-01-01', year_string + '-12-31').first().clip(aoi)
    
    #Create mask
    mangrove_year = extent_year_clipped.eq(1)
    
    #Calculate pixel area in hectares
    pixel_area_ha = ee.Image.pixelArea().divide(10000)
    
    area_img_year = pixel_area_ha.updateMask(mangrove_year)
    
    #Get total area!
    area = area_img_year.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e12
    )
    
    #Return
    return area.getInfo().get('area')

In [20]:
def GMWPercent(aoi, year):
    Hectares = GMWHectares(aoi, year)
    return (Hectares / (aoi.geometry().area(1).getInfo()/10000))*100

In [57]:
def export_GMW_tif(aoi, year, filename=None):
    
    year_string = str(year)
    extent_raster = ee.ImageCollection("projects/earthengine-legacy/assets/projects/sat-io/open-datasets/GMW/extent/GMW_V3")
    
    #Filter by year, clip to AOI:
    extent_year_clipped = extent_raster.filterDate(year_string + '-01-01', year_string + '-12-31').first().clip(aoi)
    
    #Create mask
    mangrove_year = extent_year_clipped.eq(1)
    
    #Choose filename if none
    if filename is None:
        timestamp = datetime.now().strftime('%Y_%m_%d')
        filename = f'GMW_{year}_{timestamp}.tif'
    
    
    geemap.ee_export_image(
        mangrove_year.updateMask(mangrove_year),   # only mangrove pixels
        filename='outputs/'+filename,   # output file in outputs folder
        scale=30,   #30m resolution
        region=aoi.geometry(),
        file_per_band=False
    )
    

In [30]:
print(str(GMWHectares(aoi, 2015)) + " hectares of mangroves in the PA in the year 2015.")
print(str(GMWPercent(aoi, 2015)) + "% of the area is covered in mangroves.")


72962.5311871273 hectares of mangroves in the aoi in the year 2015.
14.084726827666373% of the area is covered in mangroves.


In [56]:
export_GMW_tif(aoi, 2015)

mangrove_2015_2026_01_07.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\ajfar\Calyx\outputs\mangrove_2015_2026_01_07.tif


## JAXA section:

Testing section to visualize:

In [9]:
#Insert year you want below - JAXA year:
year = 2010
year_string = str(year)

#NOTE - JAXA DATA MISSING FROM 2011-2014
dataset_jaxa = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF')
    .filterDate(year_string + '-01-01', year_string + '-12-31')
    .filterBounds(aoi)
)

forest_non_forest = dataset_jaxa.select('fnf')
first = forest_non_forest.first().clip(aoi)
mask = first.eq(1)

m = geemap.Map()
m.centerObject(aoi, 8)
m.addLayer(aoi, {}, 'aoi')
m.addLayer(mask, {'palette': ['white', 'green']}, 'Forest mask')


m

Map(center=[17.204640472242097, -88.6973939340803], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
year = 2017
year_string = str(year)

#Starting 2017, new dataset!
dataset_jaxa = (
    ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4')
    .filterDate(year_string + '-01-01', year_string + '-12-31')
    .filterBounds(aoi)
)

forest_non_forest = dataset_jaxa.select('fnf')
first = forest_non_forest.first().clip(aoi)

vis = {
    'min': 1,
    'max': 4,
    'palette': ['00b200', '83ef62', 'ffff99','0000ff']  # dark green, light green
}

m = geemap.Map()
m.centerObject(aoi, 8)
m.addLayer(aoi, {}, 'aoi')
m.addLayer(first, vis, 'Forest mask 1 and 2 only')


m

Map(center=[17.204640472242097, -88.6973939340803], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
def JAXA_hectares_FNF3(aoi, year):
    year_str = str(year)

    # Load JAXA FNF dataset
    jaxa = (
        ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF')
        .filterDate(year_str + '-01-01', year_str + '-12-31')
        .filterBounds(aoi)
        .select('fnf')
    )

    # Get yearly image and clip
    fnf = jaxa.first().clip(aoi)

    # Forest mask (1 = forest)
    forest = fnf.eq(1)

    # Pixel area in hectares
    pixel_area_ha = ee.Image.pixelArea().divide(10000)

    # Mask to forest pixels only
    forest_area_img = pixel_area_ha.updateMask(forest)

    # Sum area
    area = forest_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=25,        # JAXA PALSAR resolution (~25 m)
        maxPixels=1e12
    )

    # Return value as Python float
    return area.getInfo().get('area')

In [13]:
def JAXA_hectares_FNF4(aoi, year):
    '''
    
    Returns DICTIONARY with keys 'Dense', "Non-dense" and "Total"
    '''
    year_string = str(year)
    
    #Load JAXA FNF4 dataset, since year >=2017
    dataset_jaxa = (
        ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4')
        .filterDate(year_string + '-01-01', year_string + '-12-31')
        .filterBounds(aoi)
        .select('fnf')
    )
    
    fnf = dataset_jaxa.first().clip(aoi)
    
    # Pixel area in hectares
    pixel_area_ha = ee.Image.pixelArea().divide(10000)
    
    #Calculate Dense forest (band 1)
    dense_mask = fnf.eq(1)
    dense_area = pixel_area_ha.updateMask(dense_mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=25,
        maxPixels=1e12
    ).get('area').getInfo()
    
    #Calculate Non-dense forest (band 2)
    nondense_mask = fnf.eq(2)
    nondense_area = pixel_area_ha.updateMask(nondense_mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=25,
        maxPixels=1e12
    ).get('area').getInfo()
    
    total_ha = dense_area + nondense_area
    
    return {
        "Dense": dense_area,
        "Non-dense": nondense_area,
        "Total": total_ha
    }

In [14]:
def JAXA_hectares(aoi, year):
    if year >= 2017:
        return JAXA_hectares_FNF4(aoi,year)
    else:
        return JAXA_hectares_FNF3(aoi,year)

In [15]:
def JAXA_percent(aoi, year):
    retVal = JAXA_hectares(aoi, year)
    #This can either turn an integer or a dictionary - check the type!
    if type(retVal) is dict:
        return {
            "Dense": round((retVal['Dense'] / (aoi.geometry().area(1).getInfo()/10000))*100, 2),
            "Non-dense": round((retVal['Non-dense'] / (aoi.geometry().area(1).getInfo()/10000))*100, 2),
            "Total": round((retVal['Total'] / (aoi.geometry().area(1).getInfo()/10000))*100, 2)
        }
  
    else:
        return round((retVal / (aoi.geometry().area(1).getInfo()/10000))*100, 2)

In [16]:
def export_JAXA_tif_FNF3(aoi, year, filename=None):
    
    year_string = str(year)
    
    # Load JAXA FNF3 dataset
    dataset_jaxa = (
        ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF')
        .filterDate(f'{year_string}-01-01', f'{year_string}-12-31')
        .filterBounds(aoi)
        .select('fnf')  # single band FNF3
    )
    
    # Clip the image to AOI
    fnf_img = dataset_jaxa.first().clip(aoi)
    
    # Optional: mask to only forest pixels (1 and 2)
    forest_mask = fnf_img.eq(1)
    
    # Default filename with timestamp if not provided
    if filename is None:
        timestamp = datetime.now().strftime('%Y_%m_%d')
        filename = f'JAXA_FNF3_{year}_{timestamp}.tif'
    
    # Export using geemap
    geemap.ee_export_image(
        fnf_img.updateMask(forest_mask),  # mask non-forest pixels
        filename=f'outputs/{filename}',          # export path
        scale=25,                                # JAXA resolution ~25m
        region=aoi.geometry(),
        file_per_band=False
    )

In [17]:
def export_JAXA_tif_FNF4(aoi, year, filename=None):
    
    year_string = str(year)
    
    #
    dataset_jaxa = (
        ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4')
        .filterDate(f'{year_string}-01-01', f'{year_string}-12-31')
        .filterBounds(aoi)
        .select('fnf')  # single band FNF3
    )
    
    fnf_img = dataset_jaxa.first().clip(aoi)
    
    #Choose filename if none
    if filename is None:
        timestamp = datetime.now().strftime('%Y_%m_%d')
        filename = f'JAXA_FNF4_{year}_{timestamp}.tif'
    print(filename)    
    
    #SOMETHING IS MISSING HERE!
    geemap.ee_export_image(
        fnf_img,   # only mangrove pixels
        filename='outputs/'+filename,   # output file in outputs folder
        scale=25,   #30m resolution
        region=aoi.geometry(),
        file_per_band=True
    )

In [18]:
def export_JAXA_tif(aoi, year, filename=None):
    if year >= 2017:
        export_JAXA_tif_FNF4(aoi,year, filename)
    else:
        export_JAXA_tif_FNF3(aoi,year, filename)

In [19]:
forest_cover = JAXA_percent(aoi, 2018)
print("Forest Cover Percentages (JAXA FNF4):")
for category, value in forest_cover.items():
    print(f"  {category:10s}: {value:6.2f}%")

Forest Cover Percentages (JAXA FNF4):
  Dense     :  58.67%
  Non-dense :  16.35%
  Total     :  75.01%


In [174]:
export_JAXA_tif(aoi, 2017, 'FNF4Testnew.tif')

FNF4Testnew.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\ajfar\Calyx\outputs\FNF4Testnew.tif


## Murray Section:

In [24]:
#Load dataset and clip
murray_dataset = ee.Image('JCU/Murray/GIC/global_tidal_wetland_change/2019').clip(aoi);

#Select relevant bands
lossBand = murray_dataset.select('loss');
lossYear = murray_dataset.select('lossYear');

# Define mask for years 2007–2017
recentLossMask = lossYear.gte(0).And(lossYear.lte(9));

# Apply mask to 'loss' band
periodLoss = lossBand.updateMask(recentLossMask);

# Simplify geometry for stable export (works for FeatureCollections too)
simplePA = aoi.geometry().simplify(100);

# Visualize
m3 = geemap.Map()
m3.centerObject(aoi, 9);
#m3.setOptions('SATELLITE');
m3.addLayer(ee.Image().paint(aoi, 0, 2),{}, 'AOI')
# m3.addLayer(
#   periodLoss,
#   {'palette': ['fe4a49'], 'min': 1, 'max': 1},
#   'Tidal wetland loss (2018–2019)'
# );
m3.addLayer(lossBand, {}, "lossband")
m3.addLayer(lossYear, {'palette' : 'red'}, "lossyear")
m3



Map(center=[24.170135459525337, 67.69670285085533], controls=(WidgetControl(options=['position', 'transparent_…

In [25]:
loss_hist = lossYear.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=aoi,
    scale=30,          # or dataset resolution
    maxPixels=1e13
)

print('Loss band value histogram:', loss_hist.getInfo())

Loss band value histogram: {'lossYear': {'10': 13669.6, '13': 10555.905882352941, '16': 14682.72156862745, '19': 14404.039215686274, '4': 12125.917647058823, '7': 11032.749019607843}}


In [28]:
def Murray_hectares_year_range(aoi, year_start, year_end):

    #Load dataset
    murray_dataset = ee.Image('JCU/Murray/GIC/global_tidal_wetland_change/2019').clip(aoi) #Don't clip

    #Select relevant bands
    lossBand = murray_dataset.select('loss');
    lossYear = murray_dataset.select('lossYear');

    #Define mask for range of years we desire
    #Note - 2000 subtracted since the key in Murray counts only the last 2 digits of year - for example,
    year_start_murray = year_start - 2000
    year_end_murray = year_end - 2000
    time_loss_mask = lossYear.gte(year_start_murray).And(lossYear.lte(year_end_murray));

    periodLoss = lossBand.eq(1)
    periodLoss = periodLoss.updateMask(time_loss_mask);

    #simplify for easy calculation

    #Calculation pixel area in hectares
    pixel_area_ha = ee.Image.pixelArea().divide(10000)

    area_img_year = pixel_area_ha.updateMask(periodLoss)

    #Get total area - reduce over geometry

    area = area_img_year.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=10, #
        maxPixels=1e13,
    )

    return ee.Number(area.get('area')).getInfo()
    


In [ ]:
def Murray_hectares(aoi, year):
    '''
    Input start year and aoi, and function returns 2 pieces of info in a dictionary
    OUTPUT: a dictionary containing 2 pieces of info: loss overall, and loss over last 10 years.
    '''
    total = Murray_hectares_year_range(aoi,1999,2019)
    ten_year = Murray_hectares_year_range(aoi,year-9,year)
    return {
        'ten_year_loss' : ten_year,
        'total' : total
    }

In [ ]:
def Murray_percent(aoi, year):
    Hectares = Murray_hectares(aoi, year)
    return {
        'ten_year_loss_percent' : (Hectares['ten_year_loss'] / (aoi.geometry().area(1).getInfo()/10000))*100,
        'total_loss_percent' : (Hectares['total'] / (aoi.geometry().area(1).getInfo()/10000))*100
    }



In [ ]:
loss_2010_2013 = Murray_Hectares_single_year(aoi, 1999, 2013)
print(f"Forest loss 2010-2013: {loss_2010_2013:.2f} ha")